[View in Colaboratory](https://colab.research.google.com/github/CSTOMJason/Cnn_cat_and_people_classification/blob/master/%E5%AD%97%E6%AF%8D%E5%9B%BE%E7%89%87_cnn_%E5%88%86%E7%B1%BB.ipynb)

In [53]:
!pip install imageio
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import tarfile
from IPython.display import display, Image
from sklearn.linear_model import LogisticRegression
from six.moves.urllib.request import urlretrieve
from six.moves import cPickle as pickle

# Config the matplotlib backend as plotting inline in IPython
%matplotlib inline

In [54]:
#下载数据集
url = 'https://commondatastorage.googleapis.com/books1000/'
last_percent_reported = None
data_root = '.' # Change me to store data elsewhere

def download_progress_hook(count, blockSize, totalSize):
  """A hook to report the progress of a download. This is mostly intended for users with
  slow internet connections. Reports every 5% change in download progress.
  """
  global last_percent_reported
  percent = int(count * blockSize * 100 / totalSize)

  if last_percent_reported != percent:
    if percent % 5 == 0:
      sys.stdout.write("%s%%" % percent)
      sys.stdout.flush()
    else:
      sys.stdout.write(".")
      sys.stdout.flush()
      
    last_percent_reported = percent
        
def maybe_download(filename, expected_bytes, force=False):
  """Download a file if not present, and make sure it's the right size."""
  dest_filename = os.path.join(data_root, filename)
  if force or not os.path.exists(dest_filename):
    print('Attempting to download:', filename) 
    filename, _ = urlretrieve(url + filename, dest_filename, reporthook=download_progress_hook)
    print('\nDownload Complete!')
  statinfo = os.stat(dest_filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified', dest_filename)
  else:
    raise Exception(
      'Failed to verify ' + dest_filename + '. Can you get to it with a browser?')
  return dest_filename

train_filename = maybe_download('notMNIST_large.tar.gz', 247336696)
test_filename = maybe_download('notMNIST_small.tar.gz', 8458043)

Found and verified ./notMNIST_large.tar.gz
Found and verified ./notMNIST_small.tar.gz


In [55]:
#解压数据集并读取数据到对应的文件夹中
num_classes = 10
np.random.seed(133)

def maybe_extract(filename, force=False):
  root = os.path.splitext(os.path.splitext(filename)[0])[0]  # remove .tar.gz
  if os.path.isdir(root) and not force:
    # You may override by setting force=True.
    print('%s already present - Skipping extraction of %s.' % (root, filename))
  else:
    print('Extracting data for %s. This may take a while. Please wait.' % root)
    tar = tarfile.open(filename)
    sys.stdout.flush()
    tar.extractall(data_root)
    tar.close()
  data_folders = [
    os.path.join(root, d) for d in sorted(os.listdir(root))
    if os.path.isdir(os.path.join(root, d))]
  if len(data_folders) != num_classes:
    raise Exception(
      'Expected %d folders, one per class. Found %d instead.' % (
        num_classes, len(data_folders)))
  print(data_folders)
  return data_folders
  
train_folders = maybe_extract(train_filename)
test_folders = maybe_extract(test_filename)

./notMNIST_large already present - Skipping extraction of ./notMNIST_large.tar.gz.
['./notMNIST_large/A', './notMNIST_large/B', './notMNIST_large/C', './notMNIST_large/D', './notMNIST_large/E', './notMNIST_large/F', './notMNIST_large/G', './notMNIST_large/H', './notMNIST_large/I', './notMNIST_large/J']
./notMNIST_small already present - Skipping extraction of ./notMNIST_small.tar.gz.
['./notMNIST_small/A', './notMNIST_small/B', './notMNIST_small/C', './notMNIST_small/D', './notMNIST_small/E', './notMNIST_small/F', './notMNIST_small/G', './notMNIST_small/H', './notMNIST_small/I', './notMNIST_small/J']


In [56]:
image_size = 28  # Pixel width and height.
pixel_depth = 255.0  # Number of levels per pixel.

def load_letter(folder, min_num_images):
  """Load the data for a single letter label."""
  image_files = os.listdir(folder)
  dataset = np.ndarray(shape=(len(image_files), image_size, image_size),
                         dtype=np.float32)
  print(folder)
  num_images = 0
  for image in image_files:
    image_file = os.path.join(folder, image)
    try:
      image_data = (imageio.imread(image_file).astype(float) - 
                    pixel_depth / 2) / pixel_depth
      if image_data.shape != (image_size, image_size):
        raise Exception('Unexpected image shape: %s' % str(image_data.shape))
      dataset[num_images, :, :] = image_data
      num_images = num_images + 1
    except (IOError, ValueError) as e:
      print('Could not read:', image_file, ':', e, '- it\'s ok, skipping.')
    
  dataset = dataset[0:num_images, :, :]
  if num_images < min_num_images:
    raise Exception('Many fewer images than expected: %d < %d' %
                    (num_images, min_num_images))
    
  print('Full dataset tensor:', dataset.shape)
  print('Mean:', np.mean(dataset))
  print('Standard deviation:', np.std(dataset))
  return dataset
        
def maybe_pickle(data_folders, min_num_images_per_class, force=False):
  dataset_names = []
  for folder in data_folders:
    set_filename = folder + '.pickle'
    dataset_names.append(set_filename)
    if os.path.exists(set_filename) and not force:
      # You may override by setting force=True.
      print('%s already present - Skipping pickling.' % set_filename)
    else:
      print('Pickling %s.' % set_filename)
      dataset = load_letter(folder, min_num_images_per_class)
      try:
        with open(set_filename, 'wb') as f:
          pickle.dump(dataset, f, pickle.HIGHEST_PROTOCOL)
      except Exception as e:
        print('Unable to save data to', set_filename, ':', e)
  
  return dataset_names

train_datasets = maybe_pickle(train_folders, 45000)
test_datasets = maybe_pickle(test_folders, 1800)

./notMNIST_large/A.pickle already present - Skipping pickling.
./notMNIST_large/B.pickle already present - Skipping pickling.
./notMNIST_large/C.pickle already present - Skipping pickling.
./notMNIST_large/D.pickle already present - Skipping pickling.
./notMNIST_large/E.pickle already present - Skipping pickling.
./notMNIST_large/F.pickle already present - Skipping pickling.
./notMNIST_large/G.pickle already present - Skipping pickling.
./notMNIST_large/H.pickle already present - Skipping pickling.
./notMNIST_large/I.pickle already present - Skipping pickling.
./notMNIST_large/J.pickle already present - Skipping pickling.
./notMNIST_small/A.pickle already present - Skipping pickling.
./notMNIST_small/B.pickle already present - Skipping pickling.
./notMNIST_small/C.pickle already present - Skipping pickling.
./notMNIST_small/D.pickle already present - Skipping pickling.
./notMNIST_small/E.pickle already present - Skipping pickling.
./notMNIST_small/F.pickle already present - Skipping pi

In [57]:
def make_arrays(nb_rows, img_size):
  if nb_rows:
    dataset = np.ndarray((nb_rows, img_size, img_size), dtype=np.float32)
    labels = np.ndarray(nb_rows, dtype=np.int32)
  else:
    dataset, labels = None, None
  return dataset, labels

def merge_datasets(pickle_files, train_size, valid_size=0):
  num_classes = len(pickle_files)
  valid_dataset, valid_labels = make_arrays(valid_size, image_size)
  train_dataset, train_labels = make_arrays(train_size, image_size)
  vsize_per_class = valid_size // num_classes
  tsize_per_class = train_size // num_classes
    
  start_v, start_t = 0, 0
  end_v, end_t = vsize_per_class, tsize_per_class
  end_l = vsize_per_class+tsize_per_class
  for label, pickle_file in enumerate(pickle_files):       
    try:
      with open(pickle_file, 'rb') as f:
        letter_set = pickle.load(f)
        # let's shuffle the letters to have random validation and training set
        np.random.shuffle(letter_set)
        if valid_dataset is not None:
          valid_letter = letter_set[:vsize_per_class, :, :]
          valid_dataset[start_v:end_v, :, :] = valid_letter
          valid_labels[start_v:end_v] = label
          start_v += vsize_per_class
          end_v += vsize_per_class
                    
        train_letter = letter_set[vsize_per_class:end_l, :, :]
        train_dataset[start_t:end_t, :, :] = train_letter
        train_labels[start_t:end_t] = label
        start_t += tsize_per_class
        end_t += tsize_per_class
    except Exception as e:
      print('Unable to process data from', pickle_file, ':', e)
      raise
    
  return valid_dataset, valid_labels, train_dataset, train_labels
            
            
train_size = 200000
valid_size = 10000
test_size = 10000

valid_dataset, valid_labels, train_dataset, train_labels = merge_datasets(
  train_datasets, train_size, valid_size)
_, _, test_dataset, test_labels = merge_datasets(test_datasets, test_size)

print('Training:', train_dataset.shape, train_labels.shape)
print('Validation:', valid_dataset.shape, valid_labels.shape)
print('Testing:', test_dataset.shape, test_labels.shape)

Training: (200000, 28, 28) (200000,)
Validation: (10000, 28, 28) (10000,)
Testing: (10000, 28, 28) (10000,)


In [0]:
def randomize(dataset, labels):
  permutation = np.random.permutation(labels.shape[0])
  shuffled_dataset = dataset[permutation,:,:]
  shuffled_labels = labels[permutation]
  return shuffled_dataset, shuffled_labels
train_dataset, train_labels = randomize(train_dataset, train_labels)
test_dataset, test_labels = randomize(test_dataset, test_labels)
valid_dataset, valid_labels = randomize(valid_dataset, valid_labels)

In [59]:
print("train_dataset.shape is",train_dataset.shape)
print("train_labels.shape is",train_labels.shape)
print("test_dataset.shape is",test_dataset.shape)
print("test_label.shape is",test_labels.shape)
print("valid_dataset.shape is",valid_dataset.shape)
print("valid_labels.shape is",valid_labels.shape)

train_dataset.shape is (200000, 28, 28)
train_labels.shape is (200000,)
test_dataset.shape is (10000, 28, 28)
test_label.shape is (10000,)
valid_dataset.shape is (10000, 28, 28)
valid_labels.shape is (10000,)


In [0]:
pickle_file = os.path.join(data_root, 'notMNIST.pickle')

try:
  f = open(pickle_file, 'wb')
  save = {
    'train_dataset': train_dataset,
    'train_labels': train_labels,
    'valid_dataset': valid_dataset,
    'valid_labels': valid_labels,
    'test_dataset': test_dataset,
    'test_labels': test_labels,
    }
  pickle.dump(save, f, pickle.HIGHEST_PROTOCOL)
  f.close()
except Exception as e:
  print('Unable to save data to', pickle_file, ':', e)
  raise

In [61]:
statinfo = os.stat(pickle_file)
print('Compressed pickle size:', statinfo.st_size)

Compressed pickle size: 690800506


In [0]:
import pickle

In [0]:
#读取.pickle的文件本数据中是保存的字母图片的矩阵的保存方式
with open("notMNIST.pickle",'rb') as f:
  data=pickle.load(f)

In [64]:
data.keys()

dict_keys(['train_dataset', 'train_labels', 'valid_dataset', 'valid_labels', 'test_dataset', 'test_labels'])

In [65]:
#数据的读取
train_x,train_y=data["train_dataset"],data["train_labels"]
print("train_x.shape is",train_x.shape)
print("train_y.shape is",train_y.shape)
valid_x,valid_y=data["valid_dataset"],data["valid_labels"]
print("valid_x.shape is",valid_x.shape)
print("valid_y.shape is",valid_y.shape)
test_x,test_y=data["test_dataset"],data["test_labels"]
print("test_x.shape is",test_x.shape)
print("test_y.shape is",test_y.shape)


train_x.shape is (200000, 28, 28)
train_y.shape is (200000,)
valid_x.shape is (10000, 28, 28)
valid_y.shape is (10000,)
test_x.shape is (10000, 28, 28)
test_y.shape is (10000,)


In [0]:
train_xx=[]
for i in range(train_x.shape[0]):
  train_xx.append(np.expand_dims(train_x[i],axis=-1))
train_xx=np.array(train_xx)

In [0]:
valid_xx=[]
for i in range(valid_x.shape[0]):
  valid_xx.append(np.expand_dims(valid_x[i],axis=-1))
valid_xx=np.array(valid_xx)

In [0]:
test_xx=[]
for i in range(test_x.shape[0]):
  test_xx.append(np.expand_dims(test_x[i],axis=-1))
test_xx=np.array(test_xx)

In [0]:
def make_labe(train_yy):
  temp=np.eye(10)
  train_lab_y=[]
  for i in range(train_yy.shape[0]):
    if train_yy[i]==0:
      train_lab_y.append(temp[0])
    elif train_yy[i]==1:
      train_lab_y.append(temp[1])
    elif train_yy[i]==2:
      train_lab_y.append(temp[2])
    elif train_yy[i]==3:
      train_lab_y.append(temp[3])
    elif train_yy[i]==4:
      train_lab_y.append(temp[4])
    elif train_yy[i]==5:
      train_lab_y.append(temp[5])
    elif train_yy[i]==6:
      train_lab_y.append(temp[6])
    elif train_yy[i]==7:
      train_lab_y.append(temp[7])
    elif train_yy[i]==8:
      train_lab_y.append(temp[8])
    elif train_yy[i]==9:
      train_lab_y.append(temp[9])
  
  train_lab_y=np.array(train_lab_y)
  return train_lab_y
    
    

In [0]:
train_yy=make_labe(train_y)

In [0]:
test_yy=make_labe(test_y)

In [0]:
import tensorflow as tf
import numpy as np


In [73]:
print("train_xx.shape is",train_xx.shape)
print("train_yy.shape is",train_yy.shape)
print("valid_xx.shape is",valid_xx.shape)
print("valid_y.shape is",valid_y.shape)
print("test_xx.shape is",test_xx.shape)
print("test_yy.shape is",test_yy.shape)


train_xx.shape is (200000, 28, 28, 1)
train_yy.shape is (200000, 10)
valid_xx.shape is (10000, 28, 28, 1)
valid_y.shape is (10000,)
test_xx.shape is (10000, 28, 28, 1)
test_yy.shape is (10000, 10)


In [0]:
def weight(shape):
  init=tf.random_normal(shape,stddev=0.1)
  return tf.Variable(init)
def biase(shape):
  init=tf.random_normal(shape,stddev=0.1)
  return tf.Variable(init)
def conv2(x,w):
  return tf.nn.conv2d(x,w,strides=[1,1,1,1],padding="SAME")
def max_pool(x):
  return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding="SAME")
def dropout(x,keep):
  return tf.nn.dropout(x,keep)
  

In [0]:
x=tf.placeholder(tf.float32,[None,28,28,1])
y=tf.placeholder(tf.float32,[None,10])
keep1=tf.placeholder(tf.float32)
keep10=tf.placeholder(tf.float32)

In [76]:
#the first layer
w1=weight([4,4,1,32])
b1=biase([32])
wb1=tf.nn.relu(conv2(x,w1)+b1)
pool1=max_pool(wb1)
drop1=dropout(pool1,keep1)
print("drop1.shape is",drop1.shape)

drop1.shape is (?, 14, 14, 32)


In [77]:
#the second layer
w2=weight([4,4,32,512])
b2=biase([512])
wb2=tf.nn.relu(conv2(drop1,w2)+b2)
pool2=max_pool(wb2)
drop2=dropout(pool2,keep1)
print("drop2.shape is",drop2.shape)

drop2.shape is (?, 7, 7, 512)


In [78]:
#full layer
wf=weight([7*7*512,128])
bf=biase([128])
drop2_flat=tf.reshape(drop2,[-1,7*7*512])
wbf=tf.nn.relu(tf.matmul(drop2_flat,wf)+bf)
dropf=dropout(wbf,keep10)
print("dropf.shape",dropf.shape)

dropf.shape (?, 128)


In [79]:
#out layer
wout=weight([128,10])
bout=biase([10])
out=tf.matmul(dropf,wout)+bout
print("out.shape is",out.shape)

out.shape is (?, 10)


In [0]:
cross_entropy=cross_entropy=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=out,labels=y))

In [0]:
train=tf.train.AdamOptimizer(0.001).minimize(cross_entropy)

In [0]:
accuracy=tf.reduce_mean(tf.cast(tf.equal(tf.argmax(out,1),tf.argmax(y,1)),tf.float32))

In [83]:
batch_size=100
num_batch=train_xx.shape[0]//batch_size
print(num_batch)

2000


In [84]:
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  for j in range(2):
    for i in range(num_batch):
      batch_x=train_xx[i*batch_size:(i+1)*batch_size]
      batch_y=train_yy[i*batch_size:(i+1)*batch_size]
      _,loss=sess.run([train,cross_entropy],feed_dict={x:batch_x,y:batch_y,keep1:0.5,keep10:0.8})
      if i%100==0:
        print("the loss is===>",loss)
    if (j+1)*batch_size%100==0:
      print("[++++++++++++++++++++++++++++++++++++++++++++++++++]")
      acc = accuracy.eval({x:test_xx, y:test_yy, keep1:1.0, keep10:1.0})
      print("the acc is ",acc)
      print("[++++++++++++++++++++++++++++++++++++++++++++++++++]")
            
      

the loss is===> 15.349357
the loss is===> 1.731937
the loss is===> 1.0850391
the loss is===> 0.9187096
the loss is===> 1.1825658
the loss is===> 0.64177096
the loss is===> 0.6987003
the loss is===> 0.4761429
the loss is===> 0.57452387
the loss is===> 0.79943544
the loss is===> 0.5283295
the loss is===> 0.46956563
the loss is===> 0.5257047
the loss is===> 0.38737392
the loss is===> 0.56285423
the loss is===> 0.4723314
the loss is===> 0.5431472
the loss is===> 0.48570648
the loss is===> 0.43629703
the loss is===> 0.71163446
[++++++++++++++++++++++++++++++++++++++++++++++++++]
the acc is  0.9408
[++++++++++++++++++++++++++++++++++++++++++++++++++]
the loss is===> 0.46446407
the loss is===> 0.5771772
the loss is===> 0.42424697
the loss is===> 0.5101562
the loss is===> 0.71990913
the loss is===> 0.33665973
the loss is===> 0.37915444
the loss is===> 0.33808655
the loss is===> 0.3828899
the loss is===> 0.67326975
the loss is===> 0.30043292
the loss is===> 0.34873176
the loss is===> 0.40842855

In [145]:
train_xx.shape

(200000, 28, 28, 1)

In [139]:
train_yy.shape

(200000, 10)

In [140]:
test_xx.shape

(10000, 28, 28, 1)

In [141]:
test_yy.shape

(10000, 10)

In [46]:
while True:
  number=input("请输入一个三位的整数")
  try:
    eval(number)
  except:
    print("输入的数字不合法")
    break
  if len(number) !=3:
    print("输入的数字不是三位数请继续")
    continue
  a=[]
  for per in number:
    a.append(pow(int(per),3))
  Sum=sum(a)
  if Sum==int(number):
    print("%d 是一个水仙花数!"%(int(number)))
  else:
    print("%d不是水仙花数"%(int(number)))
  

请输入一个三位的整数153
153 是一个水仙花数!
请输入一个三位的整数111
111不是水仙花数
请输入一个三位的整数123
123不是水仙花数
请输入一个三位的整数111
111不是水仙花数
请输入一个三位的整数asd
输入的数字不合法


In [49]:
import random
rand=random.randint(1,5)
counter=0
while True:
  gust=input("请输入1～5的任意的一个数!")
  try:
    eval(gust)
  except:
    print("数字不合法请重新输入！")
    continue
  if int(gust)<1 or int(gust)>5:
    print("输入的数字超出范围请重新输入")
    continue
  if int(gust)==rand:
    print("恭喜了才对了！")
    break
  else:
    if int(gust)<rand:
      print("猜的数字小了!")
    else:
      print("猜的数字大了!")
  

请输入1～5的任意的一个数!4
猜的数字大了!
请输入1～5的任意的一个数!2
猜的数字小了!
请输入1～5的任意的一个数!3
恭喜了才对了！
